In [0]:
%run
/Workspace/Repos/h20240186@pilani.bits-pilani.ac.in/spark-olist-pipeline/src/utils/config

In [0]:
from pyspark.sql.functions import date_format

def ingest_to_bronze(table, cfg):
    """Ingest data from source to bronze table."""
    df=spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(cfg["input_path"])
   
    if cfg["partition_col"]:
        df = df.withColumn("order_month", date_format(df[cfg["partition_col"]], "yyyy-MM"))
        df=df.repartition(50,"order_month")
        df.write.format("delta")\
            .partitionBy("order_month")\
            .option("maxRecordsPerFile", 1000000)\
            .mode("overwrite")\
            .save(cfg["output_path"])
    else:
        df.write.format("delta")\
        .mode("overwrite")\
        .option("maxRecordsPerFile", 1000000)\
        .save(cfg["output_path"])
    


In [0]:
for table, cfg in bronze_sources.items():
    ingest_to_bronze(table, cfg)